In [1]:
#importing all required libraries
import numpy as np 
import pandas as pd 
import seaborn as sns

from sklearn.linear_model import LogisticRegression 


from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
Sc = StandardScaler()

from sklearn.preprocessing import LabelEncoder 
LE=LabelEncoder()

from sklearn.model_selection import train_test_split

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [1]:
#Importing the dataset
train=pd.read_csv('/kaggle/input/titanic/train.csv')
test=pd.read_csv('/kaggle/input/titanic/test.csv')

In [1]:
#let us have look at columns of dataset
train.head()

Let us Observe the Training Dataset first.

In [1]:
#let us find the total number ofrows and columns. 
train.shape

In [1]:
#stats of passangers and on numeric cols
train.describe()
#lets us know that some age col are missing..
#also gives us more info about the price(fare)..



Give detailed Information about the dataset.

In [1]:
#lets find total count of survivors      
#0-> died ,1-> survived
train['Survived'].value_counts()

In [1]:
#showcase the count of survivours..
sns.countplot(train['Survived'])

In [1]:
#count of survivers for the counts for columns 'sex','pclass','sibSp','parch','embarked'

cols = ['Sex','Pclass','SibSp','Parch','Embarked','Fare']
n_rows=2
n_cols=3
#There will be total 5 valid plots but as the number of rows and columns are 2 and 3 respectively we will get an index out of bound warning.

#subplot grid and figure size of each graph
fig,axs = plt.subplots(n_rows,n_cols,figsize = (n_cols*3.2,n_rows*3.2))
for r in range(0,n_rows):
    for c in range(0,n_cols):
        i =r*n_cols + c#index to go through no of cols
        ax = axs[r][c]#shows to where to position subplot
        sns.countplot(train[cols[i]],hue = train['Survived'], ax = ax)
        ax.set_title(cols[i])
        ax.legend(title='survived',loc = "best")

plt.tight_layout()

In [1]:
#looking at survival rate by sex
train.groupby("Sex")[['Survived']].mean()

In [1]:
#look at survival rate by class and sex
train.pivot_table('Survived',index = 'Sex',columns = "Pclass").plot()

In [1]:
#plot the survival rate of each class
sns.barplot(x = 'Pclass',y = 'Survived',data = train)

This graph gives a brief information stating that there were higher chances of survival of passangers with first class Tickets and then following the second class and at last 3 class with vcery few chances of survival.

In [1]:
#Look up for prices of each class
plt.scatter(train['Fare'],train['Pclass'],color = 'purple',label = 'Passanger paid')
plt.ylabel('Class')
plt.xlabel('price/Fare')
plt.title('Price  of each class ')
plt.legend()
plt.show()

Above Graph Give a brief information about classifaction of passangers accordning to their class and also displays the price/Fare paind by Passangers for particulat class.
All Passangers who paid Fare 100 pounds and above would certainly  be placed in first class.
So as a result he women in the first class had a higher chance of surviving as compared to 2nd & 3rd class compartments.
This information will be much helpful whle predecting the survival

In [1]:
#count the empty values in each col
train.isna().sum()

we can observe there are lots of value missing for Age and cabin.
Such missing values affects the tranning of model and results in false prediction.
So it becomes necessary to choose carefully what key features would to take to train our model

In [1]:
train['Age'].fillna(train['Age'].median(), inplace = True)

Filling up the missing values with help of other values from the dataset.

In [1]:
#drop the cols 
train=train.drop(['Cabin','Ticket','Name','PassengerId','Embarked'],axis=1)


It is necessary to filter out the unwanted data such as their cabin,ticket number,passanger name and passanger id.
Also its very important to minize the error rate and increase accuracy of our prediction. 

In [1]:
# count new number of rows and cols in dataset
train.shape

In [1]:
#look at datatype
train.dtypes

In [1]:
train.shape

These are the key features that have been selected for our model.
We can observe that we have Sex colums with data type as object as it contains values such as Male And Female.So in next steps we convert them into unique numeric values.

In [1]:
#print the unique values in cols
print(train['Sex'].unique())


In [1]:
#transforming object datatype into int
train.iloc[:,2] = LE.fit_transform(train.iloc[:, 2].values)

Here we convert columns with object datatypes  into Integers to avoid any values bound error

In [1]:
#print the unique values in cols after transformation
#1 -> Male, 0->female
print(train['Sex'].unique())

In [1]:
#spliting the data into  independent X and dependent Y
X =train.iloc[:,1:7].values
Y =train.iloc[:,0].values 

In [1]:
#spliting dataset
X_tr,X_te,Y_tr,Y_te =train_test_split(X,Y,test_size = 0.2,random_state=0)

Spliting out Train dataser into 80% train data and 20% test data.
Spliting is been carried out just to check the accuracy of multiple models and find out which of them has the highest pergentage of accuracy which in futurte can be used to make our predictions.

In [1]:
#Function creation to check various Machine Learning Models
def models(X_tr,Y_tr):
    
   
    
    #using kNN(K nearest neighbours).
    from sklearn.neighbors import KNeighborsClassifier
    Knn = KNeighborsClassifier(n_neighbors = 5,metric = 'minkowski', p=2)
    Knn.fit(X_tr,Y_tr)
    
    #using Decision tree clissifer
    from sklearn.tree import DecisionTreeClassifier
    tree = DecisionTreeClassifier(criterion = 'entropy',random_state = 0)
    tree.fit(X_tr,Y_tr)
    
    #using Random Forest
    from sklearn.ensemble import RandomForestClassifier
    forest = RandomForestClassifier(n_estimators = 300,criterion = 'entropy',random_state=0,max_depth=1,oob_score=True)
    forest.fit(X_tr,Y_tr)
    
    #printing the training accuracy for each module
    
    print('[0] K Nearest Neighbors',Knn.score(X_tr,Y_tr))
    print('[1] Decision Tree',tree.score(X_tr,Y_tr))
    print('[2] Random Forest',forest.score(X_tr,Y_tr))
    
    return Knn,tree,forest
    

In [1]:
#get and train all models
model = models(X_tr,Y_tr)

Here we can take a look on our training accuracy of various models defined in the function.

In [1]:
#show the confusion matrix and accuracy of all models on test data
for i in range(len(model)):
    cm = confusion_matrix(Y_te,model[i].predict(X_te))
    
    #extraction True Negative , False Positive , False Negative ,True Positive
    TN,FP,FN,TP =cm.ravel()
    
    test_score = (TP+TN)/(TP+TN+FN+FP)
    
    print(cm)
    print('Model[{}] Testing Accuracy = "{}"'.format(i,test_score))
    print()
    

Here we can have lok at the confusion matrix created after prediction done on test data and also the accuracy of the particulat Model.
This helps us in choosing  Correct Model for Prediction.
Here we can see that Random Forest Classifier has the highest accuracy in predicting out test data thus using same Model for making Prediction on Test-Dataset. 

In [1]:
#print the pridction of the Logistic Regression
pred = model[2].predict(X_te)
print(pred)
print("*" * 100)

#print the actual values
print(Y_te)

In [1]:
# Checking what are the data types in test dataset
test.dtypes


In [1]:
test.isna().sum()

Finding out any Null values in Test dataset.

In [1]:
#Adding the missing vaues  using the dataset
test['Age'].fillna(test['Age'].median(), inplace = True)
test['Fare'].fillna(test['Fare'].median(), inplace = True)

Filling up the missing values and replacing their values from the avaliable data from dataset.

In [1]:
test.describe()

In [1]:
#filtering the test data according to important features
test.isna().sum()

#saving passanger id to diffrent variable for future reference
ids = test['PassengerId']

#drop the columns according to Tranning dataset.
test=test.drop(['Cabin','Ticket','PassengerId','Name','Embarked'],axis=1)


In [1]:
test.dtypes

In [1]:
#converting object datatype into Int64
test.iloc[:,1] = LE.fit_transform(test.iloc[:,1].values)


In [1]:
test.shape

In [1]:
# Assigning values to prediction variable
xte = test.iloc[:,0:6].values

Now here we interact with actual Test-Dataset.
Our dataset contains some unwanted data so we now filter out that data and keep only the key features  that were used to train our Model earlier.

In [1]:
#prediction of test dataset

pred = model[2].predict(xte)

print(pred)
print('*' * 100)
for i in pred:
    if(i==0):
        print('Dead')
    else:
        print('Survived')


Here we do the prediction of our Test-Dataset on our selected model.
This states that in the data at particular key points the passanger would have survived or would have been dead by displaying Survived or Dead 

In [1]:
#ploting the prediction
sns.countplot(pred)


Graphical representation of the number of Survivers and Dead passangers after prediction


In [1]:
#importing gender_submission to check the accuracy of predictions.
check = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')
check=check.drop(['PassengerId'],axis=1)

In [1]:
#Accuracy check
cm = confusion_matrix(check,model[2].predict(xte))
TN,FP,FN,TP =cm.ravel()
test_score = (TP+TN)/(TP+TN+FN+FP)
print(cm)
print("Accuracy of our prediction usingRandom forest is",test_score)

* * **The Accuracy of the prediction is totally based on proper selection of Key features**

In [1]:
#Lets check the Accuracy of our predictions using other models too.
#[0] logistic regressing
#[1] K Nearest Neighbors
#[2] Decision Tree
#[3] Random Forest

for i in range(len(model)):
    cm = confusion_matrix(check,model[i].predict(xte))
    
    #extraction True Negative , False Positive , False Negative ,True Positive
    TN,FP,FN,TP =cm.ravel()
    
    test_score = (TP+TN)/(TP+TN+FN+FP)
    
    print(cm)
    print('Model[{}] Testing Accuracy = "{}"'.format(i,test_score))
    print()
    

* **Here we can observe the by using Logistic Regression we can get the highest Accuracy of prediction**
* This is the order of testing accuracy from Highest to Lowest 
> 1. **logistic regressing - 0.992822966507177**
> 2. **K Nearest Neighbors - 0.937799043062201**
> 3. **Random Forest - 0.9210526315789473** 
> 4. **Decision Tree - 0.8995215311004785**

In [1]:
#Let us Save our Predictions into CSV File.
#using the passangerids which were kept aside previously.
output = pd.DataFrame({ 'PassengerId' : ids, 'Survived': y_pred })
output.to_csv('submissio.csv', index=False)
sub = pd.read_csv("submissio.csv")
sub.head()

In [1]:
from sklearn.metrics import accuracy_score

C_param_range = [0.001,0.01,0.1,1,10,100]
j = 0
for i in C_param_range:
    
    # Apply logistic regression model to training data
    lr = LogisticRegression(penalty = 'none', C = i,random_state = 0,max_iter=1000)
    lr.fit(X_tr,Y_tr)
    
    # Predict using model
    y_pred = lr.predict(xte)
    
    # Saving accuracy score in table
    #st = accuracy_score(Y_te,y_pred)
    j += 1
    

In [1]:
y_pred

In [1]:
tm = confusion_matrix(check,pred)
test_score = (TP+TN)/(TP+TN+FN+FP)
TN,FP,FN,TP =tm.ravel()
print(tm)
print("Accuracy of our prediction using Logstic Regression is",test_score)